In [1]:
import pandas as pd
import utm
import numpy as np
import random
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
import h2o
from h2o.automl import H2OAutoML
from utils.from_latlon import from_latlon
from utils.preprocessing import preprocess
from utils.postprocessing import postprocess_data, location_averaging, calculate_error

seed = 38 # Seed for train/val split

C:\Users\JohnvanOsta\AppData\Local\Temp\ipykernel_14612\3249771646.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Specify data paths
train_data = 'SBTF_data/input_data/train_test_data/train_data.xlsx'
test_data = 'SBTF_data/input_data/train_test_data/test_data.xlsx'
radio_tower_xy_path = 'SBTF_data/input_data/radio_tower_locations/RTEastNorth_1group.xlsx'

# Variable parameters
freq = '3min' # Frequency of data

# Fixed parameters
routine = 'training'
dimensions = ['xOffset', 'yOffset']


In [3]:
def preprocess_sim_data(sim_data, freq, tower_locs, routine):
    
    sim_dat_filt, predictors = preprocess(sim_data, freq, routine)
     
    # Calculate easting and northing from lat long
    sim_dat_filt['easting'], sim_dat_filt['northing'], sim_dat_filt['zone_num'], sim_dat_filt['zone_letter'] = from_latlon(sim_dat_filt['POINT_Y'].values, sim_dat_filt['POINT_X'].values)

    # Create a dictionary of the coordinates of the towers
    offset_dict = tower_locs.set_index('TowerID').to_dict()
    point_x = offset_dict['POINT_X']
    point_y = offset_dict['POINT_Y']

    # Standardise the coordinates so that the tower location == 0 on both the x and y axes.
    sim_dat_filt['xOffset'] = sim_dat_filt['easting'] - sim_dat_filt['TowerID'].map(point_x).fillna(0)
    sim_dat_filt['yOffset'] = sim_dat_filt['northing'] - sim_dat_filt['TowerID'].map(point_y).fillna(0)
    
    return sim_dat_filt, predictors


In [4]:
# Get training data
train_data = pd.read_excel(train_data)
train_data['DateAndTime'] = pd.to_datetime(train_data['DateAndTime'])

# Get testing data
test_data = pd.read_excel(test_data)
test_data['DateAndTime'] = pd.to_datetime(test_data['DateAndTime'])

# Get tower locations
tower_locs = pd.read_excel(radio_tower_xy_path)

In [5]:
train_data_preproc, predictors_train = preprocess_sim_data(train_data, freq, tower_locs, routine)


c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x0000012FC5DCB600> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])


In [6]:
len(train_data_preproc['Point_ID'].unique().tolist())

185

In [7]:
# Preprocess the training and testing data
train_data_preproc, predictors_train = preprocess_sim_data(train_data, freq, tower_locs, routine)
test_data_preproc, predictors_test = preprocess_sim_data(test_data, freq, tower_locs, routine)

unique_ids = train_data_preproc['Point_ID'].unique().tolist()

# Initialise h2o
h2o.init(nthreads = 2)

# For loop to sample training data, train, and test the models and export the result
train_data_length = len(unique_ids)
n = 5
run = 1

while n <= train_data_length:
    # Create a random subset of the training dataframe based on random Point_IDs, with the number of points equal to n
    random.shuffle(unique_ids)
    subset = unique_ids[:n]
    train_data_preproc_sample = train_data_preproc[train_data_preproc['Point_ID'].isin(subset)]

    test_data_preproc_n = test_data_preproc.copy()
    results = []

    # Train, save and test the models for each dimension
    for dimension in dimensions:
        # print(f"Training model for {dimension}")
        # Train the model
        variables = predictors_train + [dimension]
        training_input = train_data_preproc_sample[variables]
        train = h2o.H2OFrame(training_input)
        aml = H2OAutoML(max_models=20, seed=seed, stopping_metric='MAE', sort_metric='MAE', max_runtime_secs=600)
        aml.train(x=predictors_train, y=dimension, training_frame=train)
        print(aml.leaderboard)

        # Save the leader model
        # h2o.save_model(aml.leader, path = f"Example_data\Output\Trained_models\{dimension}", force=True)

        # Make predictions on the test data
        test = h2o.H2OFrame(test_data_preproc_n)
        preds = aml.leader.predict(test)

        # Save predictions to a new column in the test dataframe
        pred_column_name = f"{dimension}_pred"
        test_data_preproc_n[pred_column_name] = preds.as_data_frame()

    # Post process the test predictions to calculate location from the radio tower locations
    test_predictions_tower = postprocess_data(test_data_preproc_n, tower_locs)

    # Location averaging functions
    test_location_estimates = location_averaging(test_predictions_tower)
    test_location_estimates = calculate_error(test_location_estimates)

    # Calculate the mean absolute error of UTM_predictions['distance'] and the standard error
    mean_error = np.mean(test_location_estimates['error_m'])
    std_error = stats.sem(test_location_estimates['error_m'])

    results.append({'r': run, 'n': n, 'mean_error': mean_error, 'std_error': std_error})
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'SBTF_data/ml4rt_output/predictions/sample_size_analysis_fingerprinting/{n}_{run}_error.csv', index=False)

    print(f'Mean error (+/-SE) = {mean_error} (+/- {std_error})')
    
    n = n+5

# Stop h2o
h2o.cluster().shutdown()


Checking whether there is an H2O instance running at http://localhost:54321.

c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x0000012FC5DCB600> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])
c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\utils\preprocessing.py:10: FutureWarning: The provided callable <function std at 0x0000012FC5DCB600> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  .agg(['mean', 'count', np.std])


.... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.10+11-LTS-240, mixed mode, sharing)
  Starting server from C:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\JOHNVA~1\AppData\Local\Temp\tmpkzkvn4sg
  JVM stdout: C:\Users\JOHNVA~1\AppData\Local\Temp\tmpkzkvn4sg\h2o_JohnvanOsta_started_from_python.out
  JVM stderr: C:\Users\JOHNVA~1\AppData\Local\Temp\tmpkzkvn4sg\h2o_JohnvanOsta_started_from_python.err
  Server is running at http://127.0.0.1:54339
Connecting to H2O server at http://127.0.0.1:54339 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Australia/Brisbane
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.3
H2O_cluster_version_age:,3 months and 6 days
H2O_cluster_name:,H2O_from_python_JohnvanOsta_sos2qr
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.963 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
20:34:32.448: AutoML: XGBoost is not available; skipping it.


20:34:33.277: _min_rows param, The dataset size is too small to split for min_rows=100.0: must have at least 200.0 (weighted) rows, but have only 32.0.

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                     mae     rmse     mse    rmsle    mean_residual_deviance
DeepLearning_grid_3_AutoML_1_20240326_203432_model_1     216.21   374.306  140105      nan                    140105
DeepLearning_grid_2_AutoML_1_20240326_203432_model_1     234.521  397.167  157741      nan                    157741
StackedEnsemble_BestOfFamily_1_AutoML_1_20240326_203432  237.376  392.702  154215      nan                    154215
StackedEnsemble_AllModels_1_AutoML_1_20240326_203432     242.819  400.715  160573      nan                    160573
DeepLear

...

c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\h2o\frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
02:35:38.667: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                         mae     rmse     mse    rmsle    mean_residual_deviance
GBM_grid_1_AutoML_73_20240327_23538_model_4  267.833  398.265  158615      nan                    158615
GBM_2_AutoML_73_20240327_23538               273.453  404.931  163969      nan                    163969
GBM_grid_1_AutoML_73_20240327_23538_model_1  273.495  402.917  162342      nan                    162342
GBM_grid_1_AutoML_73_20240327_23538_model_5  273.525  403.425  162752      nan                    162752
GBM_3_AutoML_73_20240327_23538               275.11   407.704  166222      nan                    166222
GBM_5_AutoML_73_20240327_23538               275.721  408.403  166793      nan                    166793
GBM_4_AutoML_73_20240327_2353

c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\h2o\frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
02:40:40.591: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████████████| (done) 100%
model_id                                         mae     rmse     mse    rmsle    mean_residual_deviance
GBM_3_AutoML_74_20240327_24040               285.893  429.658  184606      nan                    184606
GBM_grid_1_AutoML_74_20240327_24040_model_4  286.338  428.935  183986      nan                    183986
GBM_2_AutoML_74_20240327_24040               286.432  428.883  183941      nan                    183941
GBM_5_AutoML_74_20240327_24040               286.919  428.204  183358      nan                    183358
GBM_4_AutoML_74_20240327_24040               290.122  432.926  187425      nan                    187425
GBM_grid_1_AutoML_74_20240327_24040_model_5  292.738  428.324  183461      nan                    183461
XRT_1_AutoML_74_20240327_2404

c:\Users\JohnvanOsta\Documents\GitHub\ml4rt\.venv\Lib\site-packages\h2o\frame.py:1979: H2ODependencyWarning: converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above).
  warnings.warn("converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [ ]:
test_predictions_tower.to_excel("test_tower_predictions.xlsx", index=False)
test_location_estimates.to_excel("UTM_predictions.xlsx", index=False)

In [ ]:
# sorted_df = test_location_estimates.sort_values(by='error_m', ascending=False)
# sorted_df

In [ ]:
train_data_preproc.to_excel("train_data_preproc.xlsx", index=False)